# PROJET APPLICATION DE RECOMMANDATION DE FILMS

## PARTIE 1 : KPI

In [2]:
import os
import io
import gzip
import random
import secrets
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm
import plotly.io as pio
from sklearn import tree
import plotly.express as px
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from flask import Flask, request, render_template, session, url_for, redirect

#### chargement des datasets de type TSV

In [6]:
# scrapping des noms des fichiers
url = "https://datasets.imdbws.com/"

# contournement d'éventuels protection anti-scrapping
navigator = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

response = requests.get(url, headers={"User-Agent": navigator})
# response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Trouver tous les liens sur la page
liens = soup.find_all("a")

# création de la liste des fichiers
fichiers = []
for lien in liens:
    #  choisir uniquement les liens de type href
    nom_fichier = lien.get("href")
    #  choisir uniquement les liens se terminant par .tsv.gz
    if nom_fichier.endswith(".tsv.gz"):
        # print(nom_fichier)
        fichiers.append(nom_fichier)
print(f"\n fichiers:\n{fichiers} \n")

# Liste des noms de fichiers à télécharger
# fichiers = ['title.principals.tsv.gz',
#             'title.akas.tsv.gz',
#             'name.basics.tsv.gz',
#             'title.basics.tsv.gz',
#             'title.crew.tsv.gz',
#             'title.episode.tsv.gz',
#             'title.ratings.tsv.gz' ]


 fichiers:
['https://datasets.imdbws.com/name.basics.tsv.gz', 'https://datasets.imdbws.com/title.akas.tsv.gz', 'https://datasets.imdbws.com/title.basics.tsv.gz', 'https://datasets.imdbws.com/title.crew.tsv.gz', 'https://datasets.imdbws.com/title.episode.tsv.gz', 'https://datasets.imdbws.com/title.principals.tsv.gz', 'https://datasets.imdbws.com/title.ratings.tsv.gz'] 


nom_sans_extension :
name_basics 


nom_sans_extension :
title_akas 


nom_sans_extension :
title_basics 


nom_sans_extension :
title_crew 


nom_sans_extension :
title_episode 


nom_sans_extension :
title_principals 


nom_sans_extension :
title_ratings 



In [ ]:
# Dictionnaire pour stocker les dataframes
dfs = {}

for fichier in fichiers:
    # url = f'https://datasets.imdbws.com/{fichier}'
    response = requests.get(fichier)
    gzip_file = gzip.open(io.BytesIO(response.content), "rt", encoding="utf-8")
    df = pd.read_csv(gzip_file, sep="\t", low_memory=False)
    # Chaque dataframe porte le nom du fichier sans l'extension
    nom_sans_extension = os.path.splitext(os.path.basename(fichier))[0]
    # suppression du tz et remplacement des . par des _
    nom_sans_extension = os.path.splitext(nom_sans_extension)[0]
    nom_sans_extension = nom_sans_extension.replace(".", "_")
    print(f"\nnom_sans_extension :\n{nom_sans_extension} \n")
    dfs[nom_sans_extension] = df

    # dfs[fichier] = df

In [11]:
# liste des datafdrames
for df in dfs:
    print(f"{df} ")

name_basics 
title_akas 
title_basics 
title_crew 
title_episode 
title_principals 
title_ratings 


In [32]:
# Résumé des informations des dataframes
for df in dfs:
    print(f"\ndf :{df} ")
    your_dataframe = dfs[df]

    print(
        f"shape: {your_dataframe.shape} - list columns :{your_dataframe.columns.tolist()} \nliste des colonnes numeriques: {your_dataframe.select_dtypes(include=[np.number]).columns.tolist()} \nliste des colonnes non numeriques: {your_dataframe.select_dtypes(exclude=[np.number]).columns.tolist()} \nColonne avec des na :\n{your_dataframe.isna().sum()} "
    )


    # print(f"\ndf head :\n{your_dataframe.head(2)} \n")


    # print(f"\ndf describe :\n{your_dataframe.describe()} \n")
    print(
        f"Noms des colonnes avec au moins une valeur NA : {your_dataframe.columns[your_dataframe.isna().any()].tolist()}"
    )
    print(
        f"Nombre de lignes avec au moins une valeur NA : {your_dataframe.isna().any(axis=1).sum()}"
    )


df :name_basics 
shape: (13144850, 6) - list columns :['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'] 
liste des colonnes numeriques: [] 
liste des colonnes non numeriques: ['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'] 
Colonne avec des na :
nconst                     0
primaryName                7
birthYear                  0
deathYear                  0
primaryProfession    2630039
knownForTitles             0
dtype: int64 
Noms des colonnes avec au moins une valeur NA : ['primaryName', 'primaryProfession']
Nombre de lignes avec au moins une valeur NA : 2630045

df :title_akas 
shape: (38349425, 8) - list columns :['titleId', 'ordering', 'title', 'region', 'language', 'types', 'attributes', 'isOriginalTitle'] 
liste des colonnes numeriques: ['ordering'] 
liste des colonnes non numeriques: ['titleId', 'title', 'region', 'language', 'types', 'attributes', 'isOriginalTitle'] 
Colonne avec des na :
t

In [38]:
# création d'un fichier de synthése des dataframes au format markdown
import datetime

# date et heure actuelles
now = datetime.datetime.now()

# Formatage date et heure au format 2024-01-01-9h30
date_time = now.strftime("%Y-%m-%d-%Hh%M")

# Ajouter la date et l'heure au nom du fichier
filename = f"synthése_dataframes_{date_time}.md"

with open(filename, "w", encoding="utf-8") as f:
    for df in dfs:
        your_dataframe = dfs[df]
        f.write(f"\n## DataFrame : {df}\n")
        f.write(f"- **Shape** : `{your_dataframe.shape}`\n")
        f.write(f"- **Liste des colonnes** : `{your_dataframe.columns.tolist()}`\n")
        f.write(
            f"- **Liste des colonnes numériques** : `{your_dataframe.select_dtypes(include=[np.number]).columns.tolist()}`\n"
        )
        f.write(
            f"- **Liste des colonnes non numériques** : `{your_dataframe.select_dtypes(exclude=[np.number]).columns.tolist()}`\n"
        )
        f.write(
            f"- **Colonnes avec des NA** :\n```\n{your_dataframe.isna().sum()}\n```\n"
        )
        f.write(
            f"- **Noms des colonnes avec au moins une valeur NA** : `{your_dataframe.columns[your_dataframe.isna().any()].tolist()}`\n"
        )
        f.write(
            f"- **Nombre de lignes avec au moins une valeur NA** : `{your_dataframe.isna().any(axis=1).sum()}`\n"
        )

In [ ]:
# création d'un fichier de synthése des dataframes au format markdown
import datetime
# date et heure actuelles
now = datetime.datetime.now()
# Formatage date et heure au format 2024-01-01-9h30
date_time = now.strftime(
)
# Ajouter la date et l'heure au nom du fichier
filename = f"synthése_dataframes_{date_time}.md"
with open(filename, "w", encoding="utf-8") as f:
    for df in dfs:
        your_dataframe = dfs[df]
        f.write(f"\n## DataFrame : {df}\n")
        f.write(f"- **Shape** : `{your_dataframe.shape}`\n")
        f.write(f"- **Liste des colonnes** : `{your_dataframe.columns.tolist()}`\n")
        f.write(f"- **Liste des colonnes numériques** : `{your_dataframe.select_dtypes(include=[np.number]).columns.tolist()}`\n")
        f.write(f"- **Liste des colonnes non numériques** : `{your_dataframe.select_dtypes(exclude=[np.number]).columns.tolist()}`\n")
        f.write(f"- **Colonnes avec des NA** :\n```\n{your_dataframe.isna().sum()}\n```\n")
        f.write(f"- **Noms des colonnes avec au moins une valeur NA** : `{your_dataframe.columns[your_dataframe.isna().any()].tolist()}`\n")
        f.write(f"- **Nombre de lignes avec au moins une valeur NA** : `{your_dataframe.isna().any(axis=1).sum()}`\n")